In [1]:
import numpy as np
import itertools
import scipy.stats
from scipy import sparse
import random

In [2]:
########### Cases ########

case = 'simple'

if case == 'simple':
    
    servers = 3
    M1 = 4
    M2 = 6
    max_epoch_1 = 2
    max_epoch_2 = 2
    state_dim =  2 + max_epoch_1 + max_epoch_2 - 2


    P_1_stays = 0.8
    P_2_stays = 0.08
    lambda1 = 0.25
    lambda2 = 0.4

    # The binomial probability distributions
    def g_1(s_prime, a):
        return scipy.stats.binom.pmf(s_prime[2], a[0], P_1_stays)

    def g_2(s_prime, a):
        return scipy.stats.binom.pmf(s_prime[3], a[1], P_2_stays)


    # the poisson distributions of the sources
    def f_1(s_prime, a, s):
        l = lambda1 * (M1 - a[0] -s[2])
        return scipy.stats.poisson.pmf(s_prime[0], l)

    def f_2(s_prime, a, s):
        l = lambda2 * (M2 - a[1] - s[3])
        return scipy.stats.poisson.pmf(s_prime[1], l)


    # we multiply the distribution functions
    def transition(s_prime: np.array, s: np.array, a: np.array):

        if (a[0], a[1]) not in get_legal_actions(s) and not is_legal_state(s, s_prime, a):

            return 0

        return f_1(s_prime, a, s) * f_2(s_prime, a, s) * g_1(s_prime, a) * g_2(s_prime, a)
    
    
    state = np.zeros(shape=(state_dim,), dtype=np.int)
    action = np.zeros(shape=(2,), dtype = np.int)
    
    # we order the states
    ordered_states = [np.array([s_1,s_2,s_3,s_4]) for s_1,s_2,s_3,s_4 in \
                  itertools.product(range(M1+1),range(M2+1),range(servers+1),range(servers+1)) \
                 if s_3+s_4 <= servers]

elif case == 'complicated':
    
    servers = 7
    M1 = 8
    M2 = 12
    max_epoch_1 = 4
    max_epoch_2 = 3
    state_dim =  2 + max_epoch_1 + max_epoch_2 - 2
    
###################################

    # P_source_epochs : prob that it stays
    P_1_1 = 1-1/4
    P_1_2 = 1-1/3
    P_1_3 = 1-1/2
    
    P_2_1 = 1-3/5
    P_2_2 = 1-3/4
    
    
########################

    lambda1 = 0.25
    lambda2 = 0.4

    def g_1_1(s_prime, a):
        return scipy.stats.binom.pmf(s_prime[2], a[0], P_1_1)
    
    def g_1_2(s_prime, s):
        return scipy.stats.binom.pmf(s_prime[3], s[2], P_1_2)
    
    def g_1_3(s_prime, s):
        return scipy.stats.binom.pmf(s_prime[4], s[3], P_1_3)

    def g_2_1(s_prime, a):
        return scipy.stats.binom.pmf(s_prime[5], a[1], P_2_1)
    
    def g_2_2(s_prime, s):
        return scipy.stats.binom.pmf(s_prime[6], s[5], P_2_2)


    def f_1(s_prime, a, s):
        l = lambda1 * (M1 - a[0] - s[2] - s[3] - s[4])
        return scipy.stats.poisson.pmf(s_prime[0], l)

    def f_2(s_prime, a, s):
        l = lambda2 * (M2 - a[1] - s[5] - s[6])
        return scipy.stats.poisson.pmf(s_prime[1], l)


    def transition(s_prime: np.array, s: np.array, a: np.array):

        if (a[0], a[1]) not in get_legal_actions(s) and not is_legal_state(s, s_prime, a):

            return 0

        return f_1(s_prime, a, s) * f_2(s_prime, a, s) \
                * g_1_1(s_prime, a) * g_1_2(s_prime, s) * g_1_3(s_prime, s) \
                * g_2_1(s_prime, a) * g_2_2(s_prime, s)


    state = np.zeros(shape=(state_dim,), dtype=np.int)
    action = np.zeros(shape=(2,), dtype = np.int)
    ordered_states = [np.array([s_1,s_2,s_3,s_4,s_5,s_6,s_7]) for s_1,s_2,s_3,s_4,s_5,s_6,s_7 in \
                  itertools.product(range(M1+1),range(M2+1),range(servers+1),range(servers+1),\
                                    range(servers+1),range(servers+1),range(servers+1)) \
                 if s_3+s_4+s_5+s_6+s_7 <= servers]


In [3]:
# It checks if it is possible to transition from state s to s_prime
def is_legal_state(s: np.array, s_prime: np.array, a: np.array) -> bool:
    
    if s[0] > M1 or s[1] > M2 or s_prime[0] > M1 - a[0] - np.sum(s[3:]) or \
            s_prime[1] > M2 - a[1] - np.sum(s[3:]) or s_prime[2] > a[0] \
            or s_prime[-(max_epoch_2-1)] > a[1]:
        return False
    
    states_of_job1 = [i for i in range(2,2+max_epoch_1-1)]
    states_of_job2 = [i for i in range(state_dim-max_epoch_2+1, state_dim-1)]
    
    for i in states_of_job1:
        if s_prime[i+1] > s[i]:
            return False
            

    for i in states_of_job2:
        if s_prime[i+1] > s[i]:
            return False
        
    if (a[0],a[1]) not in get_legal_actions(s):
        return False
    
    return True

In [4]:
# it gives all the legal actions given a state s
def get_legal_actions(s: np.array) -> list:

    jobs_in_system = np.sum(s[2:])
    allowable = []
    for a in itertools.product(range(state_dim+1),repeat=2):
        if a[0]+a[1] <= servers-jobs_in_system and a[0]<= s[0] and a[1] <= s[1]:
            if len(a) == 0:
                print('now')
            allowable.append(a)

    return allowable

In [5]:
# the cost function
def cost(s,a):
    
    return 3*(s[0]-a[0]) + (s[1]-a[1])

In [6]:
# It solves the system of equations that give the gain and bias,
# it assumes that h(s_0=(0,...,0))=0 arbitrarily
def equation_solver(Pd, rd):
    
    if case == 'simple':
    
        # A = Pd - I
        A = Pd - np.eye(Pd.shape[0])
        # We multiply A*h_n - ge with h_n[0]=0
        # Then we have a new matrix A in which the first column is all -1
        A[:,0] = -1 * np.ones(Pd.shape[0]).T

        # We solve A*x = -rd where x[0]=g and h_n[1:] = x[1:]
        # while h_n[0]=0 as we set before
        sol = np.linalg.solve(A, -rd)
        
        g = sol[0]
        h_n = np.zeros(Pd.shape[0])
        h_n[1:] = np.ravel(sol[1:])
        
        return g,h_n
    
    elif case == 'complicated':
        
        A = Pd - np.eye(Pd.shape[0])
        A[:,0] = -1 * np.ones(Pd.shape[0]).T

        sol = sparse.spsolve(A, -rd)

        g = sol[0]
        h_n = np.zeros(Pd.shape[0])
        h_n[1:] = np.ravel(sol[1:])
        
        return g,h_n
        

In [7]:
# it calculates a row of the transition probability matrix given a policy
# these are the probabilities that it will go from state s with action a to all the other states
# it is used in the argmin function below to find the action that minimizes the cost
def get_Pd_row(s, a):
    
    row = np.zeros(len(ordered_states))
    for i in range(len(ordered_states)):
        s_p = ordered_states[i]
        row[i] = transition(s_p, s, a)
    return row
        
        

In [8]:
# it gives the policy d so that argmin(rd + Pd*hn)
# we minimize because we have a cost minimization problem

# decision := list(index -> state, value -> action)
def argmin(h_n):
    
    d = []
    
    for s in ordered_states:
        
        possible_actions = get_legal_actions(s)
        min_val = np.inf
        best_action = -1
        for a in possible_actions:
            # rd + Pd*h_n
            val = cost(s,a) + get_Pd_row(s,a)@h_n
            if val < min_val:
                min_val = val
                best_action = a
        d.append(best_action)
    
    return d
            
    

In [9]:
# It calculates the transition probability matrix P and the reward vector r
def get_P_r(d):
    
    if case == 'simple':
    
        Pd = np.zeros(shape=(len(ordered_states),len(ordered_states)))
        rd = np.zeros(shape=(len(ordered_states),1))

        for i in range(Pd.shape[0]):
            for j in range(Pd.shape[1]):

                s = ordered_states[i]
                s_p = ordered_states[j]
                a = d[i]

                Pd[i,j] = transition(s_p, s, a)
                rd[i] = cost(s,a)
                
        return Pd, rd

    # we use sparse library because of memory limitations
    elif case == 'complicated':
    
        rd = np.zeros(shape=(len(ordered_states),1))

        Pd_data = []
        Pd_rows = []
        Pd_cols = []
        
        shape=(len(ordered_states),len(ordered_states))
        for i in range(shape[0]):
            for j in range(shape[1]):

                s = ordered_states[i]
                s_p = ordered_states[j]
                a = d[i]
                
                tr = transition(s_p, s, a)
                rd[i] = cost(s,a)
                
                if tr != 0:
                    Pd_data.append(tr)
                    Pd_rows.append(i)
                    Pd_cols.append(j)
                    
        Pd = sparse.coo_matrix((Pd_data, (Pd_rows,Pd_cols)),shape=(len(ordered_states),len(ordered_states)))
        
        return Pd, rd
                
    

In [10]:
# the policy iteration algorithm
def policy_iteration(d0: np.array):
    
    d_n = d0
    
    ## first iteration
    Pd, rd = get_P_r(d_n)
    print('Pd and rd generated')
    g,h_n = equation_solver(Pd,rd)
    print('equation solved')
    d_n_ = argmin(h_n)
    print('argmin calculated')
    iter = 1
    print('Iteration ', iter)
    
    while d_n_ != d_n:
        
        d_n = d_n_
        Pd, rd = get_P_r(d_n)
        g,h_n = equation_solver(Pd,rd)
        d_n_ = argmin(h_n)
        iter += 1
        print('Iteration ', iter)
#         print('the new policy is:', d_n_)
    
    return d_n
    
            

In [11]:
# Find all possible actions per state
possible_actions_per_state = [get_legal_actions(s) for s in ordered_states]

In [12]:
# initialize with the first possible action
d0 = [possible_actions_per_state[i][0] for i in range(len(ordered_states))]
p0 = policy_iteration(d0)

Pd and rd generated
equation solved


KeyboardInterrupt: 

In [ ]:
# initialize with the last possible action
d1 = [possible_actions_per_state[i][-1] for i in range(len(ordered_states))]
p1 = policy_iteration(d1)

In [ ]:
# check if they return the same
p0 == p1

In [15]:
# initialize randomly
d2 = [random.choice(possible_actions_per_state[i]) for i in range(len(ordered_states))]
p2 = policy_iteration(d2)

In [16]:
# check if they return the same
p0 == p2

In [105]:
# print the results to a file
# f = open('optimal_policy_{}.txt'.format(case), 'w')
for s, a in zip(ordered_states, p0):
    print('In state %s the optimal action is %s' %(tuple(s),a))
#     f.write('In state %s the optimal action is %s \n'%(s,a))
# f.close()

In state (0, 0, 0, 0) the optimal action is (0, 0)
In state (0, 0, 0, 1) the optimal action is (0, 0)
In state (0, 0, 0, 2) the optimal action is (0, 0)
In state (0, 0, 0, 3) the optimal action is (0, 0)
In state (0, 0, 1, 0) the optimal action is (0, 0)
In state (0, 0, 1, 1) the optimal action is (0, 0)
In state (0, 0, 1, 2) the optimal action is (0, 0)
In state (0, 0, 2, 0) the optimal action is (0, 0)
In state (0, 0, 2, 1) the optimal action is (0, 0)
In state (0, 0, 3, 0) the optimal action is (0, 0)
In state (0, 1, 0, 0) the optimal action is (0, 0)
In state (0, 1, 0, 1) the optimal action is (0, 0)
In state (0, 1, 0, 2) the optimal action is (0, 0)
In state (0, 1, 0, 3) the optimal action is (0, 0)
In state (0, 1, 1, 0) the optimal action is (0, 0)
In state (0, 1, 1, 1) the optimal action is (0, 0)
In state (0, 1, 1, 2) the optimal action is (0, 0)
In state (0, 1, 2, 0) the optimal action is (0, 1)
In state (0, 1, 2, 1) the optimal action is (0, 0)
In state (0, 1, 3, 0) the optim

In [51]:
# for latex
g = open('optimal_policy_LATEX_{}.txt'.format(case), 'w')
for s, a in zip(range(0,len(ordered_states)-2,3), range(0,len(p0)-2,3)):
    
    g.write('%s & %s & %s & %s & %s & %s \\\\ \n' % (tuple(ordered_states[s]),p0[a],\
                                                   tuple(ordered_states[s+1]),p0[a+1],tuple(ordered_states[s+2]),p0[a+2]))

g.write('%s & %s & %s & %s &  & \\\\ \n' % (tuple(ordered_states[-2]),p0[-2],\
                                               tuple(ordered_states[-1]),p0[-1]))
g.close()   


In [19]:
len(ordered_states)

350

In [21]:
len(p0)

350

In [23]:
-np.ones(5)

array([-1., -1., -1., -1., -1.])